In [7]:
import re
import pandas as pd 
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Functions for detecting eye laterality from free text operative notes

In [2]:
def leftcount(text):
    '''A  regular expression to detect variations of Left/OS and return their count'''
    lefteye=re.compile('(?i)(LEFT\s*EYE|\s+OS\s+|,\s+OS|[\s_]+L_*\s+eye\s+|(,\s+LEFT)|(LEFT,)|(_+LEFT_+))') 
    return len(lefteye.findall(text))
def rightcount(text):
    '''A regular expression to detect variations of Right/OD and return their count'''
    righteye=re.compile('(?i)(RIGHT\s*EYE|\s+OD\s+|,\s+OD|[\s_]+R_*\s+eye\s+|(,\s+RIGHT)|(RIGHT,)|(_+RIGHT_+))') 
    return len(righteye.findall(text))
def laterality(left, right): 
    '''returns 0 if laterality is right eye, 1 if laterality is left eye '''
    if left>right: 
        return 1
    if right>left: 
        return 0    
    
## Of note, this has only been tested on operative notes from a single academic center for cataract surgeries and glaucoma surgeries. These types of surgeries are routinely unilateral. 
## A simple algorithm such as this would not work on operations which are routinely or occasionally bilateral, such as oculoplastic surgery. 
## Such an algorithm would then have to detect which surgery was performed on which eye, which would require further development 

# Examples

## Example simple code

In [6]:
#example simple code for how this works 
leftcount("Diagnosis: Cataract, right eye Procedure: Cataract surgery with intraocular lens implantation, right eye")
rightcount("Diagnosis: Cataract, right eye Procedure: Cataract surgery with intraocular lens implantation, right eye")

0

2

## Example code for how this might work on a pandas dataframe containing a column of operative reports

In [16]:
#example code for how this might work on a pandas dataframe containing a column of operative reports 
df=pd.DataFrame(["Diagnosis: Cataract, right eye Procedure: Cataract surgery with intraocular lens implantation, right eye", 
 "Diagnosis: Cataract, left eye Procedure: Cataract surgery with intraocular lens implantation, left eye",
"Diagnosis: Cataract, right eye Procedure: Cataract surgery with intraocular lens implantation, left eye This is a surgeon dictated mistake"], columns=["note"])
df

,note
0,"Diagnosis: Cataract, right eye Procedure: Cata..."
1,"Diagnosis: Cataract, left eye Procedure: Catar..."
2,"Diagnosis: Cataract, right eye Procedure: Cata..."


In [17]:
df["right"]=df["note"].apply(rightcount)
df["left"]=df["note"].apply(leftcount)
df

,note,right,left
0,"Diagnosis: Cataract, right eye Procedure: Cata...",2,0
1,"Diagnosis: Cataract, left eye Procedure: Catar...",0,2
2,"Diagnosis: Cataract, right eye Procedure: Cata...",1,1


In [20]:
df["laterality"]=df[["left", "right"]].apply(lambda x: laterality(*x), axis=1) 
df

,note,right,left,laterality
0,"Diagnosis: Cataract, right eye Procedure: Cata...",2,0,0.0
1,"Diagnosis: Cataract, left eye Procedure: Catar...",0,2,1.0
2,"Diagnosis: Cataract, right eye Procedure: Cata...",1,1,NaN
